In [1]:
!pip install chromadb openai requests mistralai

import os
import requests
import chromadb
from mistralai import Mistral
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import uuid

# 1. Set up API key for Mistral
os.environ["MISTRAL_API_KEY"] = "2HIbLNjqvf3mt7sXkVPlmfzQlSy6g2IX"
api_key = os.getenv("MISTRAL_API_KEY")
model = "mistral-large-latest"

# 2. Initialize Mistral client
client = Mistral(api_key=api_key)

# 3. URLs for the text files from GitHub repository
base_url = "https://raw.githubusercontent.com/deep-stack/langchain-assignment-dataset/main/stories/"
files = [
    "a-mother.txt",
    "sorrow.txt",
    "the-lantern-keepers.txt",
    "the-poor-relations-story.txt",
    "the-schoolmistress.txt"
]

# 4. Download and concatenate content from all text files
file_content = ""
for file in files:
    download_url = f"{base_url}{file}"
    response = requests.get(download_url)
    if response.status_code == 200:
        file_content += response.text + "\n"
    else:
        print(f'Failed to download {file}. Status code: {response.status_code}')

# 5. Initialize Chroma client and check for collection
client_chroma = chromadb.Client()

# Check if the collection already exists and reuse it
if "documents_collection111" in client_chroma.list_collections():
    collection111 = client_chroma.get_collection("documents_collection111")
else:
    collection111 = client_chroma.create_collection("documents_collection111")

# 6. Create TF-IDF vectorizer to generate embeddings for documents
vectorizer = TfidfVectorizer()
embeddings = vectorizer.fit_transform([file_content]).toarray()

# 7. Generate unique ID for the document
document_id = str(uuid.uuid4())

# 8. Add document embedding to Chroma
collection111.add(
    ids=[document_id],
    documents=[file_content],
    metadatas=[{"source": "file_content"}],
    embeddings=embeddings
)

# 9. Define the user query (character name) and search for similar documents in Chroma
query = "Jon Snow"
query_embedding = vectorizer.transform([query]).toarray()

# 10. Perform similarity search in Chroma
results = collection111.query(
    query_embeddings=query_embedding,
    n_results=1
)

# 11. Check if any relevant document is found
if not results['documents']:
    print(f"No relevant documents found for character: {query}")
    relevant_document = "No relevant information found."
else:
    # 12. Extract the most relevant document content
    relevant_document = results['documents'][0]

# 13. Set up prompt for Mistral with the relevant document context
prompt = f"Using the following information, answer the query:\n{relevant_document}\n\nCharacter name: {query}\nReturn a JSON object with the following keys: name, storyTitle, summary, relations, characterType."

# 14. Generate response using Mistral API
messages = [
    {
        "role": "user",
        "content": prompt,
    }
]

chat_response = client.chat.complete(
    model=model,
    messages=messages,
    response_format={
        "type": "json_object",
    }
)

# 15. Output the response from Mistral
response_content = chat_response.choices[0].message.content
if response_content:
    print(response_content)
else:
    print(f"No response generated for character: {query}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.6 MB/s eta 0:00: